Deze code-cel begint met het importeren van de benodigde Python-bibliotheken. Hier zijn de belangrijkste elementen die in deze cel gebeuren:

- We importeren `numpy` als np voor numerieke berekeningen.
- We gebruiken de `load_iris` functie van scikit-learn om de Iris-dataset te laden. Deze dataset wordt gebruikt voor het evalueren van het neuraal netwerk.
- We importeren de `train_test_split` functie om de dataset in trainings- en testgegevens te splitsen.
- We importeren `MLPRegressor` van scikit-learn voor het bouwen van een regressie-gebaseerd meerlagig perceptron neuronaal netwerk.
- Tot slot onderdrukken we alle waarschuwingen om de uitvoer opgeruimd te houden.

In [6]:
# General imports for the hyperparameter optimization
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor

# Ignoring all warnings because it clutters the space where the results are supposed to go in to
import warnings
warnings.filterwarnings("ignore")


Deze code-cel behandelt het volgende:

- We laden de Iris-dataset met de `load_iris()` functie en slaan deze op in de `data` variabele.
- We splitsen de dataset in kenmerken `(X)` en doelwaarden `(y)`. Dit zijn de gegevens die het neurale netwerk zal gebruiken voor training.
- Vervolgens gebruiken we de `train_test_split` functie om de gegevens te verdelen in training en testsets. De trainingset bevat 80% van de gegevens en wordt gebruikt om het model te trainen, terwijl de testset 20% van de gegevens bevat en wordt gebruikt om de prestaties van het model te evalueren.

In [7]:
# We're still using the iris dataset for this assignment
data = load_iris()

# Splitting the data into training and testing data
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In deze code-cel definiëren we de hyperparameters die we willen onderzoeken met ons genetisch algoritme:

- __Activeringsfuncties__: We overwegen vier verschillende activeringsfuncties die in neurale netwerken kunnen worden gebruikt. Dit zijn `logistic`, `tanh`, `relu`, en `identity`.

- __Grootte van verborgen lagen (hidden layers)__: We beschouwen verschillende groottes voor de verborgen lagen van het neurale netwerk. Deze maten worden gedefinieerd als een lijst van getallen, bijvoorbeeld `[10, 20, 30, 40]`. Elke waarde vertegenwoordigt het aantal neuronen in de verborgen laag.

- __Oplossers (solvers)__: We overwegen drie verschillende optimalisatiealgoritmen (solvers) om de optimale gewichten in het neurale netwerk te vinden. Dit zijn `lbfgs`, `sgd`, en `adam`. We gebruiken de standaard leersnelheid voor elk van deze solvers.

Deze hyperparameters zullen door het genetisch algoritme worden geoptimaliseerd om de best mogelijke configuratie voor het neurale netwerk te vinden.

In [8]:
# Defining the hyperparameters that we want to test
activation_functions = ['logistic', 'tanh', 'relu', 'identity']
# The hidden layer sizes are defined as a list of tuples, where each tuple represents a hidden layer
hidden_layer_sizes = [10, 20, 30, 40]
# The solvers are the optimizers that are used to find the optimal weights
# We're using the default learning rate for each solver
solvers = ['lbfgs', 'sgd', 'adam']

In deze code-cel definiëren we de belangrijkste functies en het genetisch algoritme dat de optimale hyperparameters voor het regressie-gebaseerde neuraal netwerk zal vinden. Laten we het in delen bespreken:

- __evaluate_individual__: Dit is een functie die de nauwkeurigheid van een individuele configuratie van hyperparameters evalueert. We creëren een MLPRegressor met de opgegeven hyperparameters, trainen het op de trainingset en evalueren de score op de testset.

- __genetic_algorithm__: Dit is de kern van het genetische algoritme. We beginnen met het genereren van een willekeurige populatie van individuen met verschillende hyperparameters. Vervolgens herhalen we het genetische algoritme over een aantal generaties. In elke generatie selecteren we de beste individuen op basis van hun prestaties en genereren we een nieuwe populatie door kruising en mutatie. Uiteindelijk vinden we het individu met de beste hyperparameters voor het regressieneuraal netwerk.

In [9]:
# The function that evaluates the accuracy of an individual
def evaluate_individual(activation_function, hidden_layer_size, solver):
    clf = MLPRegressor(hidden_layer_sizes=(hidden_layer_size,), activation=activation_function, solver=solver, max_iter=2000, random_state=42)
    clf.fit(X_train, y_train)
    accuracy = clf.score(X_test, y_test)
    return accuracy

# The genetic algorithm that finds the best individual
def genetic_algorithm(population_size, generations):
    population = []

    # Creating the initial population
    for _ in range(population_size):
        activation_function = np.random.choice(activation_functions)
        hidden_layer_size = np.random.choice(hidden_layer_sizes)
        solver = np.random.choice(solvers)
        individual = (activation_function, hidden_layer_size, solver)
        population.append(individual)

    # Running the genetic algorithm
    for generation in range(generations):
        scores = [evaluate_individual(activation_function, hidden_layer_size, solver) for activation_function, hidden_layer_size, solver in population]
        selected_indices = np.argsort(scores)[::-1][:int(population_size * 0.2)]
        selected_population = [population[i] for i in selected_indices]

        # Creating the new population
        new_population = []
        for _ in range(population_size):
            index1, index2 = np.random.choice(len(selected_population), size=2, replace=False)
            parent1 = selected_population[index1]
            parent2 = selected_population[index2]
            parent1_activation_function, parent1_hidden_layer_size, parent1_solver = parent1
            parent2_activation_function, parent2_hidden_layer_size, parent2_solver = parent2
            child = (parent1_activation_function, parent2_hidden_layer_size, parent2_solver)
            new_population.append(child)

        # Mutating the new population
        population = new_population

    # Finding the best individual
    best_individual = max(population, key=lambda ind: evaluate_individual(ind[0], ind[1], ind[2]))
    return best_individual


Deze code-cel voert het genetisch algoritme uit en drukt de optimale hyperparameters af die zijn gevonden. De optimale hyperparameters vertegenwoordigen de beste combinatie voor het regressieneuraal netwerk dat is aangepast aan de gegeven dataset en configuratie.

In [10]:
# Running the genetic algorithm and printing the results
best_hyperparameters_regressor = genetic_algorithm(population_size=50, generations=10)
print("Optimal hyperparameters:", best_hyperparameters_regressor)

Optimale hyperparameters: ('tanh', 10, 'lbfgs')
